In [ ]:
import os
from tqdm import tqdm
from scripts.performance_calculation import *
from scripts.consensus_methods import CONSENSUS_METHODS
from rdkit.Chem import PandasTools
import os
import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [ ]:
DUDE_dir = Path('/home/tony/FINAL_RESULTS/DUD-E')

for dir in tqdm(sorted(os.listdir(DUDE_dir)), total=len(os.listdir(DUDE_dir))):
    if "DUD-E_RESULTS" not in dir:
        w_dir = str(DUDE_dir)+"/"+dir+"/receptor_protoss_prepared_protoss"
        actives_library = str(DUDE_dir)+"/"+dir+ "/merged_actives_decoys.sdf"
        percentages = [0.5, 1, 5]
        calculate_performance(Path(w_dir), Path(actives_library), percentages)

In [ ]:
# Define the main directory containing the target folders
DUDE_dir = Path('/home/tony/FINAL_RESULTS/DUD-E')

metrics = ['AUC_ROC', 'BEDROC', 'AUC', 'EF_0.5%', 'EF_1%', 'EF_5%', 'RIE']

for metric in tqdm(metrics):
    results = pd.DataFrame(columns=['clustering', 'consensus', 'scoring'])
    # Iterate through each subdirectory
    for dir in sorted(os.listdir(DUDE_dir)):
        if "DUD-E_RESULTS" not in dir:
            w_dir = str(DUDE_dir)+"/"+dir+"/receptor_protoss_prepared_protoss"
            performance_csv = Path(w_dir) / 'performance/performance.csv'
            # Check if the performance.csv file exists
            if performance_csv.exists():
                # Read the performance.csv file
                df = pd.read_csv(performance_csv)
                df = df[['clustering', 'consensus', 'scoring', metric]]
                # Extract the target name from the directory name
                target_name = dir
                df = df.rename(columns={metric: target_name})
                # Append the results to the main DataFrame
                results = pd.merge(results, df, on=['clustering', 'consensus', 'scoring'], how='outer')
    # Save the results to a CSV file
    results.to_csv(f'/home/tony/FINAL_RESULTS/DUD-E/DUD-E_RESULTS/results_dockm8_{metric}.csv', index=False)


In [ ]:
# Define the main directory containing the target folders
DUDE_dir = Path('/home/tony/FINAL_RESULTS/DUD-E')

metrics = ['AUC_ROC', 'BEDROC', 'AUC', 'EF_0.5%', 'EF_1%', 'EF_5%', 'RIE']


results_max = pd.DataFrame()
# Iterate through each subdirectory
for dir in sorted(os.listdir(DUDE_dir)):
    if "DEKOIS_RESULTS" not in dir:
        w_dir = str(DUDE_dir)+"/"+dir+"/receptor_protoss_prepared_protoss"
        performance_csv = Path(w_dir) / 'performance/performance.csv'
        # Check if the performance.csv file exists
        if performance_csv.exists():
            # Read the performance.csv file
            df = pd.read_csv(performance_csv)
            df = df.sort_values(by='EF_1%', ascending=False)
            dir_max = df.head(1)
            dir_max['Model'] = "DockM8-max"
            dir_max['pdb_id'] = str(dir)
            for metric in metrics:
                dir_max[f'max_{metric}'] = dir_max[metric].max()
                dir_max.drop(metric, axis=1, inplace=True)
            results_max = pd.concat([results_max, dir_max])


In [ ]:
results_max.to_csv('./results_dockm8_max.csv', index=False)